# Explore

Recipes to explore datasets: detangle functional dependencies and identify candidate keys.

In [1]:
import pandas as pd

## Load example datasets

In [2]:
ucdp_df = pd.read_csv("../example_datasets/source_data/ucdp-prio-acd-191.csv", encoding = 'utf-8')

In [3]:
ucdp_df.columns

Index(['conflict_id', 'location', 'side_a', 'side_a_id', 'side_a_2nd',
       'side_b', 'side_b_id', 'side_b_2nd', 'incompatibility',
       'territory_name', 'year', 'intensity_level', 'cumulative_intensity',
       'type_of_conflict', 'start_date', 'start_prec', 'start_date2',
       'start_prec2', 'ep_end', 'ep_end_date', 'ep_end_prec', 'gwno_a',
       'gwno_a_2nd', 'gwno_b', 'gwno_b_2nd', 'gwno_loc', 'region', 'version'],
      dtype='object')

In [4]:
ucdp_df.head(10)

,conflict_id,location,side_a,side_a_id,side_a_2nd,side_b,side_b_id,side_b_2nd,incompatibility,territory_name,...,ep_end,ep_end_date,ep_end_prec,gwno_a,gwno_a_2nd,gwno_b,gwno_b_2nd,gwno_loc,region,version
0,13637,Afghanistan,Government of Afghanistan,130,"Government of Pakistan, Government of United S...",IS,234,NaN,1,Islamic State,...,0,NaN,NaN,700,"770, 2",NaN,NaN,700,3,19.1
1,13637,Afghanistan,Government of Afghanistan,130,"Government of Pakistan, Government of United S...",IS,234,NaN,1,Islamic State,...,0,NaN,NaN,700,"770, 2",NaN,NaN,700,3,19.1
2,13637,Afghanistan,Government of Afghanistan,130,Government of United States of America,IS,234,NaN,1,Islamic State,...,0,NaN,NaN,700,2,NaN,NaN,700,3,19.1
3,13637,Afghanistan,Government of Afghanistan,130,Government of United States of America,IS,234,NaN,1,Islamic State,...,0,NaN,NaN,700,2,NaN,NaN,700,3,19.1
4,333,Afghanistan,Government of Afghanistan,130,NaN,PDPA,291,NaN,2,NaN,...,0,NaN,NaN,700,NaN,NaN,NaN,700,3,19.1
5,333,Afghanistan,Government of Afghanistan,130,NaN,Jam'iyyat-i Islami-yi Afghanistan,292,NaN,2,NaN,...,0,NaN,NaN,700,NaN,NaN,NaN,700,3,19.1
6,333,Afghanistan,Government of Afghanistan,130,Government of Russia (Soviet Union),"Harakat-i Inqilab-i Islami-yi Afghanistan , Hi...","293, 299, 294, 295, 292, 296",NaN,2,NaN,...,0,NaN,NaN,700,365,NaN,NaN,700,3,19.1
7,333,Afghanistan,Government of Afghanistan,130,Government of Russia (Soviet Union),"Harakat-i Inqilab-i Islami-yi Afghanistan , Hi...","293, 299, 294, 297, 295, 292, 296",NaN,2,NaN,...,0,NaN,NaN,700,365,NaN,NaN,700,3,19.1
8,333,Afghanistan,Government of Afghanistan,130,Government of Russia (Soviet Union),"Harakat-i Inqilab-i Islami-yi Afghanistan , Hi...","293, 299, 294, 297, 295, 292, 296",NaN,2,NaN,...,0,NaN,NaN,700,365,NaN,NaN,700,3,19.1
9,333,Afghanistan,Government of Afghanistan,130,Government of Russia (Soviet Union),"Harakat-i Inqilab-i Islami-yi Afghanistan , Hi...","293, 299, 294, 297, 295, 292, 296",NaN,2,NaN,...,0,NaN,NaN,700,365,NaN,NaN,700,3,19.1


## Identify Candidate Keys

The 'conflict_id' column seems like the most likely primary key. Does it satisfy the uniqueness constraint?

In [5]:
ucdp_df['conflict_id'].duplicated().sum()

2098

No. Does a combination of the 'conflict_id' and another column satisfy the uniqueness constraint?

In [6]:
ucdp_df[['conflict_id', 'year']].duplicated().sum()

0

Yes. Together, 'conflict_id' and 'year' form a composite candidate key. However, I used my prior knowledge of this dataset to form and test this hypothesis. Are there other candidate keys in this dataset as well? Let's find out.

In [7]:
columns = ucdp_df.columns.tolist()

candidate_keys = set()
# first, test all single columns
for c in columns:
    if ucdp_df[c].duplicated().sum() == 0:
        candidate_keys.add((c))
# next, test all combinations of 2 columns:
for c1 in columns:
    for c2 in columns:
        if c1 == c2:
            pass
        else:
            if ucdp_df[[c1, c2]].duplicated().sum() == 0:
                candidate_keys.add((c1, c2))

In [8]:
candidate_keys

{('conflict_id', 'year'), ('year', 'conflict_id')}

There are no other candidate keys with 1 or two prime attributes. But, let's generalize and create a function to find all candidate keys given some maximum number of attributes.

In [9]:
from itertools import combinations

In [10]:
def find_candidate_keys(df, max_columns=3):
    candidate_keys = []
    columns = df.columns.tolist()
    column_powerset = []
    # generate power set of columns
    for i in range(max_columns):
        combo = combinations(columns, i+1)
        column_powerset.extend(combo)
    # test elements in powerset
    for combo in column_powerset:
        ck = False
        if df[list(combo)].duplicated().sum() == 0:
            ck = True
            for k in candidate_keys:
                if set(combo).issuperset(set(k)):
                    ck = False
        if ck:
            candidate_keys.append(combo)
            
    return candidate_keys

In [11]:
ucdp_df.columns

Index(['conflict_id', 'location', 'side_a', 'side_a_id', 'side_a_2nd',
       'side_b', 'side_b_id', 'side_b_2nd', 'incompatibility',
       'territory_name', 'year', 'intensity_level', 'cumulative_intensity',
       'type_of_conflict', 'start_date', 'start_prec', 'start_date2',
       'start_prec2', 'ep_end', 'ep_end_date', 'ep_end_prec', 'gwno_a',
       'gwno_a_2nd', 'gwno_b', 'gwno_b_2nd', 'gwno_loc', 'region', 'version'],
      dtype='object')

In [12]:
find_candidate_keys(ucdp_df, max_columns=3)

[('conflict_id', 'year'),
 ('location', 'territory_name', 'year'),
 ('side_a', 'side_b', 'year'),
 ('side_a', 'side_b_id', 'year'),
 ('side_a_id', 'side_b', 'year'),
 ('side_a_id', 'side_b_id', 'year'),
 ('side_b', 'year', 'start_date'),
 ('side_b', 'year', 'start_date2'),
 ('side_b', 'year', 'gwno_a'),
 ('side_b_id', 'year', 'start_date'),
 ('side_b_id', 'year', 'start_date2'),
 ('side_b_id', 'year', 'gwno_a'),
 ('territory_name', 'year', 'start_date'),
 ('territory_name', 'year', 'gwno_loc')]

Now we have a list of all possible combinations of attributes (maximum of 3) that can uniquely identify an observation. It will take some knowledge of your dataset to determine the best one, but generally the best one will have the fewest number of attributes.

## Identify Attributes With No Variance

You may have noticed the column called 'version'. This is an example of dataset metadata being contained in the actual dataset - and it has absolutely no variance (same value for every row):

In [13]:
ucdp_df['version'].unique()

array([19.1])

You may have also noticed that the column 'ep_end_prec' is empty - and therefore also has no variance:

In [14]:
ucdp_df['ep_end_prec'].unique()

array([nan])

These columns can interfere with our other exploratory functions (finding candidate keys and functional dependencies). So, lets create a function to identify which columns have no variance.

In [15]:
def find_no_variance_columns(df):
    no_var_columns = []
    columns = df.columns.tolist()
    for c in columns:
        if df[c].nunique() in [0, 1]:
            no_var_columns.append(c)
    
    return no_var_columns

In [16]:
find_no_variance_columns(ucdp_df)

['ep_end_prec', 'version']

## Identify Attributes With Low Variance

Attributes with low variance may be categorical variables. For example, 'start_prec' has only 7 possible values:

In [17]:
ucdp_df['start_prec'].unique()

array([1, 2, 3, 5, 4, 7, 6])

Let's create a function that find which columns have a low variance given a certain threshold, and how many levels that potential categorical attribute has:

In [18]:
def find_low_variance_columns(df, threshold:int):
    low_var_columns = []
    columns = df.columns.tolist()
    for c in columns:
        if 1 < df[c].nunique() <= threshold:
            low_var_columns.append((c, df[c].nunique()))
    
    return low_var_columns

In [19]:
find_low_variance_columns(ucdp_df, 10)

[('incompatibility', 3),
 ('intensity_level', 2),
 ('cumulative_intensity', 2),
 ('type_of_conflict', 4),
 ('start_prec', 7),
 ('start_prec2', 7),
 ('ep_end', 2)]

## Identify Functional Dependencies

Just because 'conflict_id' doesn't uniquely identify the entire observation, doesn't mean that it can't uniquely identify some subset of the columns. Let's find out which columns can be completely determined by 'conflict_id'.

In [20]:
ucdp_df.groupby('conflict_id')['start_date'].nunique().unique().tolist()

[1]

In [21]:
ucdp_df.groupby('conflict_id')['territory_name'].nunique().unique().tolist()

[0, 1]

In [22]:
ucdp_df.groupby('conflict_id')['cumulative_intensity'].nunique().unique().tolist()

[1, 2]

'start_date' has one unique value per 'conflict_id'. 'territory_name' has either no values or one unique value per 'conflict_id'. 'cumulative_intensity' has one or two unique values per 'conflict_id'.

This means that 'start_date' and 'territory_name' are functionally dependent only on 'conflict_id', while 'cumulative_intensity' is not.

Let's generalize and create a function that will find all columns that are functionally dependent on a given column (or combination of columns).

In [23]:
def find_dependent_columns(df, key:list, drop_no_variance=True):
    dependent_columns = []
    columns = df.columns.tolist()
    if drop_no_variance:
        no_var_columns = find_no_variance_columns(df)
        if no_var_columns:
            for c in no_var_columns:
                columns.remove(c)
    for k in key:
        columns.remove(k)
    for c in columns:
        unique_elements = df.groupby(key)[c].nunique().unique().tolist()
        if (unique_elements == [1]) or (unique_elements == [0]) or (unique_elements == [1, 0]) or (unique_elements == [0, 1]):
            dependent_columns.append(c)
    
    return dependent_columns

In [24]:
find_dependent_columns(ucdp_df, ['gwno_a'])

['side_a', 'side_a_id']

In [25]:
find_dependent_columns(ucdp_df, ['conflict_id', 'start_date2'])

['location',
 'side_a',
 'side_a_id',
 'incompatibility',
 'territory_name',
 'start_date',
 'start_prec',
 'start_prec2',
 'ep_end_date',
 'gwno_a',
 'gwno_b',
 'gwno_loc',
 'region']

Now we have a list of all columns that are functionally dependent on both 'conflict_id' and 'start_date2' (episode start date). This indicates that these columns should be split off to form a new table, with 'conflict_id' and 'start_date2' as the composite primary key.